# Data Exploration

####  A common task with Pandas is data exploration - play with your data!

- Questions to ask:
    - How complete is my data?
    - What is the range of values?
    - What are the basic trends? 
<br>
<br>
- Overall goals are to determine:
    - If quality could be improved with cleaning/munging/wrangling
    - If transformation and/or aggregation is appropriate
    - How to address edge cases and rare scenarios that exist in the data
<br>
<br>
- Main Goal:
    - Determine if the data is "trustworthy"
        - Unexpected values or trends should be verified for accuracy
        - GIGO applies to data as much as code

In [1]:
# Dependencies

import pandas as pd

## Create a pandas dataframe from a dictionary

In [2]:
# Dictionary data of pet types and number owned
example_data = {'pet_type': ["dog","cat","bird"], 'number_owned': [2,3,35]}

# Convert to dataframe
example_df = pd.DataFrame.from_dict(example_data)

In [3]:
# Display results with print:

print(f"{example_df.head()}")

  pet_type  number_owned
0      dog             2
1      cat             3
2     bird            35


In [4]:
# Or run the df.head() or df.tail() directly in the notebook cell for formatted results:

example_df.head()

,pet_type,number_owned
0,dog,2
1,cat,3
2,bird,35


## Converting data formats into a pandas dataframe

### Example is data of steam games for number of players

In [5]:
# Location of data
csv_games = ("./Resources/SteamCharts.csv")

# Convert into pandas dataframe (encoding is optional, default is utf-8)
df_raw = pd.read_csv(csv_games, encoding="unicode_escape")

df_raw.head(3)

,gamename,year,month,avg,gain,peak,avg_peak_perc
0,Counter-Strike: Global Offensive,2021,February,741013.24,-2196.42,1123485,65.9567%
1,Counter-Strike: Global Offensive,2021,January,743209.66,25405.91,1124553,66.0893%
2,Counter-Strike: Global Offensive,2020,December,717803.75,49049.17,1164396,61.646%


Popular convention is to name initial dataframe "raw" and create copies during modifications to preserve original

In [6]:
# Renaming columns/fields as a copy
df = df_raw.rename(columns={"gamename": "game", 
                                              "year": "year", 
                                              "month": "month", 
                                              "avg": "avg_players",
                                              "gain": "monthly_change",
                                              "peak": "max_players", 
                                              "avg_peak_perc": "avg_max_percent"}).copy()

df.head(3)

,game,year,month,avg_players,monthly_change,max_players,avg_max_percent
0,Counter-Strike: Global Offensive,2021,February,741013.24,-2196.42,1123485,65.9567%
1,Counter-Strike: Global Offensive,2021,January,743209.66,25405.91,1124553,66.0893%
2,Counter-Strike: Global Offensive,2020,December,717803.75,49049.17,1164396,61.646%


In [7]:
# Confirm datatypes

df.dtypes

game                object
year                 int64
month               object
avg_players        float64
monthly_change     float64
max_players          int64
avg_max_percent     object
dtype: object

In [8]:
# Confirm number of rows and columns

print(f"Rows: {df.shape[0]}; Columns: {df.shape[1]}")

Rows: 83789; Columns: 7


In [9]:
# Determine which columns have missing data

df.isnull().sum()

game                  0
year                  0
month                 0
avg_players           0
monthly_change     1260
max_players           0
avg_max_percent       0
dtype: int64

#### Exploration shows that the field ```monthly_change``` is missing 1260 values - good candidate to drop or recalculate based on ```month``` and ```avg_players values```

In [10]:
# Max and min (agg will apply to string values based on alphanumeric ordering, if applicable)

df.agg([min, max])

,game,year,month,avg_players,monthly_change,max_players,avg_max_percent
min,100% Orange Juice,2012,April,0.00,-250248.99,0,0%
max,theHunter: Call of the Wild,2021,September,1584886.77,426446.12,3236027,NaN%


#### We can see that ```avg_max_percent``` has a value of NaN% - indicating a possible data error

In [11]:
# Descriptive Status

df.describe()

,year,avg_players,monthly_change,max_players
count,83789.000000,8.378900e+04,82529.000000,8.378900e+04
mean,2017.338708,2.792017e+03,-10.245559,5.518877e+03
std,2.237587,2.648480e+04,3789.944005,5.015571e+04
min,2012.000000,0.000000e+00,-250248.990000,0.000000e+00
25%,2016.000000,5.325000e+01,-38.310000,1.380000e+02
50%,2018.000000,2.042700e+02,-1.630000,5.020000e+02
75%,2019.000000,7.716500e+02,22.320000,1.741000e+03
max,2021.000000,1.584887e+06,426446.120000,3.236027e+06


### Can investigate individual columns

In [12]:
# Using dot notation to find all unique values for month

df.month.unique()

array(['February ', 'January ', 'December ', 'November ', 'October ',
       'September ', 'August ', 'July ', 'June ', 'May ', 'April ',
       'March '], dtype=object)

In [13]:
# Using quotations (useful when a space exists in column name)

df["year"].sort_values().unique()

array([2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021])

# Exploration Results

### This data is requires minimal cleaning (preprocessing)

Example of a cleaning method - replace missing (NaN) with zero for ```monthly_change``` column

In [14]:
# Convert NaN to zero
df.monthly_change = df.monthly_change.fillna(0)

df.isnull().sum()

game               0
year               0
month              0
avg_players        0
monthly_change     0
max_players        0
avg_max_percent    0
dtype: int64